# 1. Import Python Packages

In [1]:
%matplotlib notebook
import sys, os
sys.path.insert(0,'/global/homes/b/bpb/metatlas')

sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )


import metatlas.metatlas_objects as metob
from metatlas.helpers import mzmine_helpers as mzm
from metatlas.helpers import dill2plots as dp
from metatlas.helpers import metatlas_get_data_helper_fun as ma_data
from metatlas.helpers import rt_corrector as rt_corrector
from metatlas.helpers import chromatograms_mp_plots as cp

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import multiprocessing as mp

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 1000)


/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
# module unload oracle-jdk/1.7_64bit
# module unload oracle_client/11.2.0.3.0
# module load oracle-jdk/1.8_64bit
# export _JAVA_OPTIONS="-Xmx440G"

# /project/projectdirs/metatlas/projects/mzmine_parameters/MZmine-2.21/startMZmine_Headless.sh /project/projectdirs/metatlas/projects/mzmine_parameters/PseudoC_C18_NEG_job_script_parameters.xmlbpb@edison07:/project/projectdirs/metatlas/projects/mzmine_parameters>

In [ ]:
# filename = '/global/homes/b/bpb/batch_params/000008_pos_c18_dangl.xml'
# with open(filename,'r') as fid:
#     xml_str = fid.read()


In [ ]:
task = metob.MZMineTask()
task

In [ ]:
mzm = reload(mzm)
xml_str = mzm.get_batch_file_template()
d = mzm.xml_to_dict(xml_str)
for i,k in enumerate(d['batch']['batchstep']):
    print(i,k['@method'])

In [ ]:
task.polarity = 'negative'

task.min_peak_duration = 0.025
task.max_peak_duration = 30.0
task.rt_tol_perfile = 0.015
task.rt_tol_multifile = 0.15
task.min_peak_height = 1e6
task.noise_floor = 3e4
task.mz_tolerance = 10.0
task.min_sn_ratio = 2.0

task.output_csv = '/global/homes/b/bpb/Downloads/20170222_OMALLEY_FUNGUS_NEG_MZMINE_1to10.csv'
task.input_xml = '/global/homes/b/bpb/batch_params/20170222_OMALLEY_FUNGUS_NEG_MZMINE_1to10.xml'
# task.input_xml = '/project/projectdirs/metatlas/projects/mzmine_parameters/batch_files/test_batch.xml'

In [ ]:
# groups = dp.select_groups_for_analysis(name = '20151130_LS_Archetypes%C18_pos',
#                                        most_recent = True,
#                                        remove_empty = True,#'Strain=SB214'
#                                        include_list = [], exclude_list = ['QC', 'Blank','L2B56','HA26','HA21','L2B49','D1B20A','L2B47','L1B44','L2B36', 'L2B57', 'HD36','HC17', 'HD72', 'L1B56','HD65'])#['QC','Blank'])
# files = []
# for g in groups:
#     for f in g.items:
#         files.append(f)
# task.lcmsruns = files
# new_d = mzm.replace_files(d,list(pd.unique([f.mzml_file for f in task.lcmsruns])))
# list(pd.unique([f.name for f in task.lcmsruns]))

In [ ]:
files = metob.retrieve('Lcmsruns',name = '%NEG%UV_Fungus_1to10%102040%',username='*')
# files = metob.retrieve('Lcmsruns',hdf5_file = '%neg%coelicolor_media%day6%',username='*')
# files = [f for f in files if not '1to10' in f.name]
task.lcmsruns = files
new_d = mzm.replace_files(d,list(pd.unique([f.mzml_file for f in task.lcmsruns])))
list(pd.unique([f.name for f in task.lcmsruns]))

In [ ]:
task.mzmine_launcher = mzm.get_latest_mzmine_binary()
new_d = mzm.configure_mass_detection(new_d,task.noise_floor)
new_d = mzm.configure_chromatogram_builder(new_d,task.min_peak_duration,task.min_peak_height,task.mz_tolerance)
new_d = mzm.configure_peak_deconvolution(new_d,task.min_peak_height,task.min_sn_ratio,task.min_peak_duration,task.max_peak_duration)
new_d = mzm.configure_isotope_adduct_fragment_search(new_d,task.mz_tolerance,task.rt_tol_perfile,task.polarity,task.min_peak_height)
new_d = mzm.configure_join_aligner(new_d,task.mz_tolerance,task.rt_tol_multifile)
new_d = mzm.configure_duplicate_filter(new_d,task.mz_tolerance,task.rt_tol_perfile)
new_d = mzm.configure_gap_filling(new_d,task.mz_tolerance)
new_d = mzm.configure_csv_output(new_d,task.output_csv)

t = mzm.dict_to_etree(new_d)
mzm.indent_tree(t)
xml_batch_str = mzm.tree_to_xml(t,filename=task.input_xml)

In [ ]:
pd.set_option('display.max_colwidth', 200)
df = metob.to_dataframe([task])
df = df[[c for c in df.columns if not 'lcmsruns' in c]]
df.head()

In [ ]:
# todo: create scripts at nersc by editing the startmzmine_Linux script.
# todo: create in mzmine_helpers.py a start task that automatically
#     defines a unique filename for batch job and output.csv

In [ ]:
print('%s %s'%(task.mzmine_launcher,task.input_xml))

### Steps

* make fileinfo sheet
* create groups
* select groups
* __configure mzmine batch job__

```
1. replace files with selected files
2. replace ppm instances with specified ppm
3. replace retention time instances with specified ppm
4. add/remove adducts
5. generate metatlas entry for task
```

* deploy mzmine batch job
* check status of mzmine batch job
* import results of mzmine batch job
* configure analysis
* perform analysis
* select/remove rows from results based on analysis
* create atlas from results
* get data using metatlas


In [ ]:
filename = '/global/homes/b/bpb/batch_params/000008_pos_c18_dangl.xml'
with open(filename,'r') as fid:
    xml_str = fid.read()
# xml_str
e = ET.XML(xml_str)

d = etree_to_dict(e)

for k in d['batch']['batchstep']:
    print(k['@method'])

In [ ]:


print(ET.tostring(dict_to_etree(d)))

In [7]:
mzm = reload(mzm)
mzmine_output_file = '/global/u2/b/bpb/Downloads/20170222_OMALLEY_FUNGUS_NEG_MZMINE_1to10.csv'
df,original_mzmine = mzm.metatlas_formatted_atlas_from_mzmine_output(mzmine_output_file,'positive',
                                                                     make_atlas=False,min_rt=0.55,
                                                                    remove_fragments=False,
                                                                    remove_adducts=False,
                                                                    remove_clusters=False)

In [8]:
df.to_csv('/global/homes/b/bpb/Downloads/20170222_OMALLEY_FUNGUS_NEG_MZMINE_1to10_METATLAS.csv')

In [ ]:
original_mzmine.head(100)

In [ ]:
# df_super = df[[c for c in df.columns if not 'super' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'kana' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'max_intensity' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'Excontrol' in c]]
# df_super = df_super[[c for c in df_super.columns if not 'adduct' in c]]

# df_super.set_index(['label','mz','mz_tolerance','rt_peak','rt_min','rt_max','inchi_key','detected_polarity'],inplace=True)
# df_super.columns = [''.join(c.split('_')[2:3]) if 'pellet' in c else c for c in df_super.columns ]


In [ ]:
df_super = df[[c for c in df.columns if not 'pellet' in c]]
df_super = df_super[[c for c in df_super.columns if not 'kana' in c]]
df_super = df_super[[c for c in df_super.columns if not 'max_intensity' in c]]
df_super = df_super[[c for c in df_super.columns if not 'Excontrol' in c]]
df_super = df_super[[c for c in df_super.columns if not 'adduct' in c]]
df_super['rt_min'] = df_super['rt_peak'] - 0.2
df_super.set_index(['label','mz','mz_tolerance','rt_peak','rt_min','rt_max','inchi_key','detected_polarity'],inplace=True)
df_super.columns = [''.join(c.split('_')[2:3]) if 'super' in c else c for c in df_super.columns ]


In [ ]:
import numpy as np
from scipy.stats import ttest_ind

group1 = df_super[[c for c in df_super.columns if c == 'LB']]
group2 = df_super[[c for c in df_super.columns if c == 'PS']]

# group2 = df[df['group'] == 'GROUP2']['data'].astype(float)

t, p = ttest_ind(group1.T, group2.T)

stats_df = pd.DataFrame(index=df_super.index)
stats_df['p_value'] = p
stats_df['t_score'] = t
stats_df['log2_fold_change'] = group2.min(axis=1).apply(lambda x: np.log2(x+1)) - \
                                group1.max(axis=1).apply(lambda x: np.log2(x+1)) 
stats_df['in_control'] = group1.median(axis=1) > 1e5
stats_df['median_control'] = group1.median(axis=1)
stats_df['median_treatment'] = group2.median(axis=1)
stats_df.head()


In [ ]:
fig, ax = plt.subplots(figsize=(8,7))
stats_df['log2_fold_change'].hist(bins=100)
ax.set_yscale('log')
ax.set_xlabel('LOG2 Fold Change')
ax.set_ylabel('#features')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,7))
ax.plot(stats_df['log2_fold_change'],stats_df['p_value'],'.')
# stats_df['fold_change'].apply(lambda x: np.log2(x+1)).hist(bins=100)
ax.set_yscale('log')
ax.set_xlabel('LOG2 fold change')
ax.set_ylabel('p_value')
plt.show()

In [ ]:
filtered_df = stats_df[(stats_df['log2_fold_change']>7) & (stats_df['median_treatment']>1e5)]
fig, ax = plt.subplots(figsize=(8,7))
# color_map = {filtered_df.log2_fold_change.min():"r", filtered_df.log2_fold_change.max():"g"} 
# z_as_colors = map(color_map.get, filtered_df.log2_fold_change) 
sc = ax.scatter(filtered_df.index.get_level_values('mz'),
                filtered_df.index.get_level_values('rt_peak'),
                c = filtered_df.log2_fold_change)
# stats_df['fold_change'].apply(lambda x: np.log2(x+1)).hist(bins=100)
# ax.set_yscale('log')
plt.colorbar(sc)
ax.set_xlabel('mz')
ax.set_ylabel('rt')

In [ ]:
# (~stats_df['in_control']) & 
filtered_df = stats_df[(stats_df['log2_fold_change']>7) & (stats_df['median_treatment']>3e5)]
print filtered_df.shape
# pd.options.display.precision = 5
# pd.set_option('display.max',200)
filtered_df.sort_values('log2_fold_change', ascending=False).head(20)

In [ ]:
df_super.iloc[df_super.index.get_level_values('label') == '754.3994@3.75']

In [ ]:
filtered_df.head()

In [ ]:

# filtered_df.index.names
df = pd.DataFrame(columns=filtered_df.index.names)
for i,row in enumerate(filtered_df.index.get_values()):
    for j,c in enumerate(df.columns):
        df.loc[i,c] = row[j]
# df.columns = 
# filtered_df.index.get_values()
# #.reset_index(level=[0,1,2])
# atlas_df = atlas_df[atlas_df.columns[:6]]
# # atlas_df.reset_index(inplace=True)
# # atlas_df = atlas_df[[]]
# # atlas_df.reset_index(inplace=True)
# atlas_df.fillna('')
# atlas_df

In [ ]:
myAtlas = dp.make_atlas_from_spreadsheet(df,'20170406_ions_made_psimiae_in_LB_pos_c18',filetype='dataframe',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=True,
                                      mz_tolerance = 10)
atlas_df = ma_data.make_atlas_df(myAtlas)


In [ ]:

groups = dp.select_groups_for_analysis(name = '%rexmalm_pos_super%',
                                       most_recent = True,
                                       remove_empty = True,#'Strain=SB214'
                                       include_list = [], exclude_list = [],)#['QC','Blank'])


In [ ]:
for g in groups:
    for i in g.items:
        print i.name

In [ ]:
all_files = []
for my_group in groups:
    for my_file in my_group.items:
        all_files.append((my_file,my_group,atlas_df,myAtlas))
        
pool = mp.Pool(processes=min(10, len(all_files)))
# from metatlas.helpers.metatlas_get_data_helper_fun import get_data_for_atlas_df_and_file
metatlas_dataset = pool.map(ma_data.get_data_for_atlas_df_and_file, all_files)
pool.close()
pool.terminate()
#If you're code crashes here, make sure to terminate any processes left open.


In [ ]:
output_dir = '/global/homes/b/bpb/Downloads/psim_c18_important_pos_super/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
atlas_identifications = dp.export_atlas_to_spreadsheet(myAtlas,os.path.join(output_dir,'atlas_export.csv'))
# dp = reload(dp)
# atlas_identifications = dp.export_atlas_to_spreadsheet(myAtlas,'%s/sheets/%s.csv'%(plot_location_label,myAtlas.name))
peak_height = dp.make_output_dataframe(input_fname = '',input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_height' , output_loc=os.path.join(output_dir,'sheets'))
# peak_area = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='peak_area' , output_loc=os.path.join(output_dir,'sheets'))
# mz_peak = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_peak' , output_loc=os.path.join(output_dir,'sheets'))
# rt_peak = dp.make_output_dataframe(input_fname = my_file, input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [],fieldname='rt_peak' , output_loc=os.path.join(output_dir,'sheets'))
# mz_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='mz_centroid' , output_loc=os.path.join(output_dir,'sheets'))
# rt_centroid = dp.make_output_dataframe(input_fname = my_file,input_dataset = metatlas_dataset,include_lcmsruns = [],exclude_lcmsruns = [], fieldname='rt_centroid' , output_loc=os.path.join(output_dir,'sheets'))

In [ ]:
peak_height.head()

In [ ]:

peak_height.shape

In [ ]:
# from scipy import cluster
# mat = peak_height.as_matrix()
# mat[np.isnan(mat)] = np.nanmin(mat)
# labels = cluster.hierarchy.fclusterdata(mat,0.2,criterion='distance',method='average',metric='correlation')
# results = pd.DataFrame(data=labels, columns=['cluster'], index=peak_height.index) 

In [ ]:
peak_height.index = ['@'.join(c.split('_')[:2]).replace('p','.') for c in peak_height.index]

In [ ]:
peak_height.columns = peak_height.columns.droplevel(0)

In [ ]:
peak_height.columns = ['_'.join(c.split('_')[14:18]) for c in peak_height.columns]

In [ ]:
peak_height = peak_height.fillna(0)

In [ ]:
# norm_peak_height = peak_height.copy()
# norm_peak_height = norm_peak_height.apply(lambda x: x / x.max())
# norm_peak_height.head()

In [ ]:
peak_height.shape

In [ ]:
# norm_peak_height.columns.sort_values().shape

In [ ]:
# c

In [ ]:
sf = pd.DataFrame()
for c in peak_height.columns.sort_values():
    sf[c] = peak_height[c]
for i,row in sf.iterrows():
    sf.loc[i,:] = row/row.max()

In [ ]:
sf.head()

In [ ]:
cm = plt.colormaps()

for i,c in enumerate(cm):
    print i,c

In [ ]:
for i,r in sf.iterrows():
    print r.max()

In [ ]:
# The returned object has a savefig method that should be used if you want to save the figure object without clipping the dendrograms.

# To access the reordered row indices, use: clustergrid.dendrogram_row.reordered_ind

# Column indices, use: clustergrid.dendrogram_col.reordered_ind

g.dendrogram_row.reordered_ind

In [ ]:
import seaborn as sns
cmap = cm[148]
# cmap = sns.light_palette(as_cmap=True)
# cmap = sns.diverging_palette(h_neg=210, h_pos=350, s=90, l=30, as_cmap=True)
# g  =sns.clustermap(peak_height.apply(lambda x: (x+1)**0.25),metric='correlation',figsize=(25,25),cmap=cmap)
# g  =sns.clustermap(peak_height.apply(lambda x: np.log10(x+1)),metric='correlation',figsize=(25,25))
g  =sns.clustermap(sf,metric='euclidean',figsize=(30,18),cmap=cmap,col_cluster=True)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)  # For y axis
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90) # For x axis
plt.show()
# # http://seaborn.pydata.org/generated/seaborn.clustermap.html

In [ ]:
# from scipy import stats
# zscore = lambda x: (x - stats.nanmean(x)) / stats.nanstd(x)


In [ ]:
# fig, ax = plt.subplots(2,3,figsize=(20,30))
# for i in range(6):
#     plt.subplot(2,3,i+1)
#     row = peak_height.iloc[[i]]
#     c = 0
#     vals = []
#     for k,v in row.to_dict().items():
#         vals.append(v.items()[0][-1])
#     v = np.asarray(vals)
# #     v = v - np.nanmin(v)
# #     v = v / np.nanmax(v)
#     v[np.isnan(v)] = np.nanmin(v)
#     plt.plot(v,'.-')
#     plt.title(myAtlas.compound_identifications[i].name)
#     plt.gca().set_yscale('log')
# #     ax.set_yscale('log')
# plt.tight_layout()
# # for i,row in peak_height.iterrows():

    
# # peak_height.iloc[0,]